# Вебинар 8. Итоговый проект

---

## Recap по финальному проекту

**Основное**
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

In [1]:
# Внимание! При implicit==0.4.4 всё работает верно. 
# Но на более поздних версиях код может выдавать Segmentation Fault
# pip install implicit==0.4.4

In [2]:
import pandas as pd
import numpy as np
import lightgbm
import re
import warnings
from random import shuffle, choice
from lightgbm import LGBMClassifier
from pandarallel import pandarallel
from pycaret.classification import ClassificationExperiment
from pycaret.classification import load_model, predict_model

from src.tools import paralell_execution, ar_split_eq_cpu
from src.metrics import money_precision_at_k
from src.recommenders import MainRecommender
from src.utils import (reduce_mem_usage, prefilter_items, add_features,
                       get_new_user_features, get_new_item_features, 
                       get_important_features, get_final_recs, 
                       get_popularity_recommendations, get_targets, 
                       postfilter_items, cat_filter)

warnings.filterwarnings('ignore')
pandarallel.initialize(use_memory_fs=False, verbose=0)

data_main = pd.read_csv('data/retail_train.csv')
data = data_main[:int(data_main.shape[0]*0.8)]
data_test = data_main[int(data_main.shape[0]*0.8):]
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

user_features.columns = [col.lower() for col in user_features.columns]
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

N = 150 # 150 кол-во рекомендаций
VAL_SIZE = 5 # кол-во недель в валидационной выборке

data_train = data[data['week_no'] < data['week_no'].max() - (VAL_SIZE)]
val = data[data['week_no'] >= data['week_no'].max() - (VAL_SIZE)]
data_train_val = val.copy()

# Отбираем подходящие товары
n_items_before = data_train['item_id'].nunique()
data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)
n_items_after = data_train['item_id'].nunique()

print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 77059 to 5001


In [3]:
params = []

for n_factors in [25, 50, 100]:
    for iterations in [15, 25, 35]:
        for regularization in [0.001, 0.01, 0.05]:
            params.append({
                           'n_factors': n_factors, 
                           'iterations': iterations, 
                           'regularization': regularization, 
                          })
shuffle(params)
print(f'Всего тестируется параметров: {len(params)}')
print('*' * 50)

def train(params):
    p = params['PARAMS']
    param = p['main_recommender_params']
    data_train_val = p['data_train_val']
    data_train = p['data_train']
    item_features = p['item_features']
    user_features = p['user_features']
    N = p['N']
    
    # учим модель первого уровня
    recommender = MainRecommender(data_train, 
                                  weighting=True, 
                                  n_factors=param['n_factors'], 
                                  regularization=param['regularization'], 
                                  iterations=param['iterations'])
    
    # делаем эмбеддинги
    items_embeddings_df = recommender.items_embeddings_df
    users_embeddings_df = recommender.users_embeddings_df
    
    # добавляем новые фитчи
    df_train = add_features(data_train_val,
                          data_train,
                          recommender,
                          item_features, 
                          user_features,
                          items_embeddings_df,
                          users_embeddings_df,
                          N)
    df_test = add_features(data_test, 
                           data_train, 
                           recommender, 
                           item_features,   
                           user_features, 
                           items_embeddings_df, 
                           users_embeddings_df, 
                           N)
    
    # фильтруем название столбцов
    df_train = df_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    df_test = df_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

    # сжимаем данные для ускорения
    df_test = reduce_mem_usage(df_test)
    df_train = reduce_mem_usage(df_train)
    
    # категориальные переменные
    cat_feats = list(set(df_train.columns) - set(df_train._get_numeric_data().columns)) + ['user_id', 'item_id']
    # числовые переменные
    non_cat_feats = [col for col in df_train if col not in cat_feats]
    
    # Готовим тренировочную выборку
    # готовимся к машинному обучению
    X_train = df_train.drop(['target'], axis=1)
    y_train = df_train[['target']]
    
    # категориальные переменные
    X_train[cat_feats] = X_train[cat_feats].astype('category')
    
    # Готовим тестовую выборку
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test[['target']]
    X_test[cat_feats] = X_test[cat_feats].astype('category')
    
    # обучаем модель с учётом важности фитч
    model = LGBMClassifier(
        objective='binary', 
        max_depth=5, 
        categorical_column=cat_feats
    )
    important_feats = get_important_features(model, X_train, y_train)
    model.fit(X_train[important_feats], y_train)
    
    # Делаем предсказания по гоотвой модели
    test_preds_proba = model.predict_proba(X_test[important_feats])[:, 1]
    
    res = get_final_recs(X_test, test_preds_proba, data, data_train, item_features)
        
    price = data_train.groupby('item_id')['price'].mean().reset_index()
    
    money_result = res.parallel_apply(lambda row: money_precision_at_k(row['recommendations'], 
                                                              row['actual'], 
                                                              price), axis=1).mean()
    
    return {'money_result': money_result,
            'params': param,
            'model': model,
            'res': res}

args_parallel = []
for param in ar_split_eq_cpu(params):
    args_parallel.append({
        'PARAMS':{
            'main_recommender_params': param[0],
            'data_train_val': data_train_val,
            'data_train': data_train,
            'item_features': item_features,
            'user_features': user_features,
            'N': N,
        }
    })

# запускаем процесс параллельных вычислений
res = paralell_execution(func=train,
                         arg=args_parallel,
                         method='multiprocessing')

best_money_result = 0.0
best_params = {}
best_model = None
best_res = None

for row in res: # объединяем результаты со всех ядер
    if row['money_result'] > best_money_result:
        best_money_result = row['money_result']
        best_params = row['params']
        best_model = row['model']
        best_res = row['res']

print('*' * 50)
print(f'Лучшая метрика money_precision: {best_money_result} '+\
      f'при параметрах {best_params}')

Всего тестируется параметров: 27
**************************************************
**************************************************
Лучшая метрика money_precision: 0.44947690749871416 при параметрах {'n_factors': 25, 'iterations': 15, 'regularization': 0.05}


In [4]:
# Сохранение самой успешной модели
best_res.drop('actual', axis=1, inplace=True)

best_res.head(3)

best_res.to_csv('recommendations.csv', index=False)